In [102]:
import pandas as pd
import re
import csv
from IPython.display import display, clear_output
import nltk 
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
import string
from string import capwords
from sklearn.feature_extraction.text import CountVectorizer
import geograpy3
from cliff.api import Cliff
import json 


In [103]:
df = pd.read_csv("../dataframes/03_clean_tokenize.csv")

In [104]:
display(df.head())

,text,tokenized_text,no_stopwords_tokenized_text,no_stopwords_text,tdm
0,fracturing and subsidence of the land surface ...,"['fracturing', 'and', 'subsidence', 'of', 'the...","['fracturing', 'subsidence', 'land', 'surface'...",fracturing subsidence land surface caused with...,amount aquifers area artesian bench bo...
1,an analysis of instabilities caused by salinit...,"['an', 'analysis', 'of', 'instabilities', 'cau...","['analysis', 'instabilities', 'caused', 'salin...",analysis instabilities caused salinity gradien...,always amplitude analyse analysis aquif...
2,pollution of the rhine and water supply dutch ...,"['pollution', 'of', 'the', 'rhine', 'and', 'wa...","['pollution', 'rhine', 'water', 'supply', 'dut...",pollution rhine water supply dutch water suppl...,activities almost approximately article ...
3,development and in place leaching of mountain ...,"['development', 'and', 'in', 'place', 'leachin...","['development', 'place', 'leaching', 'mountain...",development place leaching mountain city chalc...,aid analyzed annualized approximately b...
4,the study of groundwater movement in boreholes...,"['the', 'study', 'of', 'groundwater', 'movemen...","['study', 'groundwater', 'movement', 'borehole...",study groundwater movement boreholes performed...,among aquifers authors avoiding based ...


In [105]:
countries_list = []
american_state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
american_state_names = [state.lower() for state in american_state_names]
candian_state_names = ['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick', 'Newfoundland and Labrador', 'Northwest Territories', 'Nova Scotia', 'Nunavut', 'Ontario', 'Prince Edward Island', 'Quebec', 'Saskatchewan', 'Yukon Territory']
candian_state_names = [state.lower() for state in candian_state_names]
indian_state_names = [ "Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jammu and Kashmir", "Jharkhand", "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", "Tripura", "Uttarakhand", "Uttar Pradesh", "West Bengal", "Andaman and Nicobar Islands", "Chandigarh", "Dadra and Nagar Haveli", "Daman and Diu", "Delhi", "Lakshadweep", "Puducherry"]
indian_state_names = [state.lower() for state in indian_state_names]


In [174]:
abbr_country_dict = {}
with open("../data/iso_list_of_countries.csv") as countries:
    reader = csv.reader(countries, delimiter=',',quotechar= '"',quoting= csv.QUOTE_MINIMAL)
    for row in reader:
        name = row[0]
        if name == "name":
            continue
        abbr = row[1]
        countries_list.append(name.lower())
        abbr_country_dict[abbr] = name.lower()

In [175]:
# print(american_state_names)
# print("****")
# print(countries_list)
# print("****")
# print(abbr_country_dict)

In [186]:
demonym_dict = {"hawaiian": "hawaii", "alaskan": "alaska", "indian": "india", "chinese": "china", "american": "america", "russian": "russia", "indonesian": "indonesia", "pakistani": "pakistan", "nigerian": "nigeria"}


In [187]:
province_to_country = {}
with open('../data/provinces.json') as json_file: 
        provinces = json.load(json_file) 
for p_dict in provinces:
    if "english" in p_dict:
        province_name = p_dict['english'].lower()
    else:
        province_name = p_dict['name'].lower()
    province_to_country[province_name] = abbr_country_dict[p_dict['country']]
    
    

In [188]:
def match_countries(text):
    countries_in_abstract = []
    for word in text.split():
        if word in countries_list and word not in countries_in_abstract:
            countries_in_abstract.append(word)
#         if word in abbr_country_dict and abbr_country_dict[word] not in countries_in_abstract:
#             countries_in_abstract.append(abbr_country_dict[word])
        if word in province_to_country and province_to_country[word] not in countries_in_abstract:
            countries_in_abstract.append(province_to_country[word])
        if word in demonym_dict and demonym_dict[word] not in countries_in_abstract:
            countries_in_abstract.append(demonym_dict[word])
    return countries_in_abstract

df['countries'] = df['text'].apply(lambda x: match_countries(x))

In [189]:
display(df.head())


,text,tokenized_text,no_stopwords_tokenized_text,no_stopwords_text,tdm,countries
0,fracturing and subsidence of the land surface ...,"['fracturing', 'and', 'subsidence', 'of', 'the...","['fracturing', 'subsidence', 'land', 'surface'...",fracturing subsidence land surface caused with...,amount aquifers area artesian bench bo...,[united states of america]
1,an analysis of instabilities caused by salinit...,"['an', 'analysis', 'of', 'instabilities', 'cau...","['analysis', 'instabilities', 'caused', 'salin...",analysis instabilities caused salinity gradien...,always amplitude analyse analysis aquif...,[]
2,pollution of the rhine and water supply dutch ...,"['pollution', 'of', 'the', 'rhine', 'and', 'wa...","['pollution', 'rhine', 'water', 'supply', 'dut...",pollution rhine water supply dutch water suppl...,activities almost approximately article ...,[united kingdom of great britain and northern ...
3,development and in place leaching of mountain ...,"['development', 'and', 'in', 'place', 'leachin...","['development', 'place', 'leaching', 'mountain...",development place leaching mountain city chalc...,aid analyzed annualized approximately b...,[united states of america]
4,the study of groundwater movement in boreholes...,"['the', 'study', 'of', 'groundwater', 'movemen...","['study', 'groundwater', 'movement', 'borehole...",study groundwater movement boreholes performed...,among aquifers authors avoiding based ...,[spain]


In [190]:
count = 0 
for country in country_list:
    if len(country) > 0:
        count +=1
print(count)
print(len(df.index))
print(count/len(df.index))
        

3452
7183
0.4805791452039538


In [191]:
def geograpy_countries(input):
    places = geograpy3.get_place_context(text = input)
    return places.countries
#df['geograpy_countries'] = df['text'].apply(lambda x: geograpy_countries(capwords(x)))
print(geograpy_countries("I like China and India"))

['China', 'British Indian Ocean Territory', 'United States', 'Russia']


In [192]:
display(df.head())

,text,tokenized_text,no_stopwords_tokenized_text,no_stopwords_text,tdm,countries
0,fracturing and subsidence of the land surface ...,"['fracturing', 'and', 'subsidence', 'of', 'the...","['fracturing', 'subsidence', 'land', 'surface'...",fracturing subsidence land surface caused with...,amount aquifers area artesian bench bo...,[united states of america]
1,an analysis of instabilities caused by salinit...,"['an', 'analysis', 'of', 'instabilities', 'cau...","['analysis', 'instabilities', 'caused', 'salin...",analysis instabilities caused salinity gradien...,always amplitude analyse analysis aquif...,[]
2,pollution of the rhine and water supply dutch ...,"['pollution', 'of', 'the', 'rhine', 'and', 'wa...","['pollution', 'rhine', 'water', 'supply', 'dut...",pollution rhine water supply dutch water suppl...,activities almost approximately article ...,[united kingdom of great britain and northern ...
3,development and in place leaching of mountain ...,"['development', 'and', 'in', 'place', 'leachin...","['development', 'place', 'leaching', 'mountain...",development place leaching mountain city chalc...,aid analyzed annualized approximately b...,[united states of america]
4,the study of groundwater movement in boreholes...,"['the', 'study', 'of', 'groundwater', 'movemen...","['study', 'groundwater', 'movement', 'borehole...",study groundwater movement boreholes performed...,among aquifers authors avoiding based ...,[spain]


In [193]:
# my_cliff = Cliff('http://localhost:8080')
# l = my_cliff.parse_text("This is about Einstien at the IIT in New Delhi.")
# print(l)

In [196]:
iter = 0 

for row in df.itertuples():
    if(iter >= 10):
        break
    if(len(row[6]) < 1):
        print(row[6])
        print(row[1])
        print("****")
        iter +=1
    

[]
an analysis of instabilities caused by salinity gradients in groundwater analyse des phnomnes dinstabilit provoqus par la prsence de gradients de salinit dans les eaux souterraines salinity gradients in groundwaters may lead to convection currents the present study concerns stability criteria of the flow field as well as an analysis of the salinity transport due to finite amplitude disturbances it was found that convection cells are always two dimensional rolls whose axes are parallel to the unperturbed velocity vector salinity transport through the aquifer is considerably increased due to the convection motion mechanical dispersion due to the convection motion reduces the intensity of salinity transport   taylor  francis group llc
****
[]
soil mechanics tests on coal mine spoils spoil placement associated with strip mine operations often leads to landslides and other environmental problems in an effort to circumvent these problems spoil materials and the strip mining process were e

In [197]:
print(df.loc[4, 'text'])

the study of groundwater movement in boreholes performed at the site for the future beninar dam spain using radioactive tracers in the study of sites for the construction of new dams it is necessary to know among other characteristics of the soil the following onesi ground impermeability at site to be sure of the resistance and stability of the dam ii basin impermeability for the stored waters avoiding uncontrollated losses a valuable information on this subject may be obtained through the study of waters movement at the saturated zone the method here described is based on marking the water column with a radioactive isotope and has been investigated and employed by the authors in more than  boreholes the purpose of thos e experiences was in most of the cases to know the behaviour of the groundwater aquifers as well as their recharge conditions exploitability and more importants parameters in similar manner has been successfully used at boreholes performed for the geological research of